- 참고: https://github.com/towhee-io/examples/blob/main/nlp/text_search/search_article_in_medium.ipynb
- 참고: https://hotorch.tistory.com/414

# Data preprocessing

## Load Data

In [26]:
hwp_file = './data/hwp_to_text.txt'
with open(hwp_file, 'r') as f:
    data = f.read()

In [34]:
data_list = [x.strip() for x in data.split("\n")]

In [378]:
find_date = data_list[2].find('시행')
open_date = data_list[2][find_date+2:find_date+15].strip()
open_date = open_date.replace(" ", "")

In [35]:
data_list

['\x02捤獥\x00\x00\x00\x00\x02\x02汤捯\x00\x00\x00\x00\x02\x0b漠杳\x00\x00\x00\x00',
 '주택도시기금 운용 및 관리규정',
 '\x10慤桥\x00\x00\x00\x00\x10\x10潴景\x00\x00\x00\x00\x10[시행 2019. 12. 31.] [국토교통부훈령 제1271호, 2019. 12. 31, 일부개정.]',
 '氠瑢\x00\x00\x00\x00',
 '주택도시기금 운용 및 관리규정',
 '氠瑢\x00\x00\x00\x00',
 '漠杳\x00\x00\x00\x00\x0b 법제처',
 '- \x12湯慴\x00\x00\x00\x00\x12 / \x12湯慴\x00\x00\x00\x00\x12 -',
 '漠杳\x00\x00\x00\x00\x0b 국가법령정보센터',
 '국토교통부(총칙,성과평가), 044-201-3338, 3347',
 '국토교통부(기금운용계획,회계), 044-201-3339, 3346',
 '국토교통부(리스크,자산유동화), 044-201-3340, 3344',
 '제1편 총칙',
 '제1조(목적) 이 규정은 「주택도시기금법」 제3조에 따라 설치된 주택도시기금의 운용 및 관리에 관하여 필요한 사항을 규정함을 목적으로 한다.',
 '제2조(적용대상) 기금의 운용 및 관리업무는 관계법령에 규정한 것을 제외하고는 이 규정이 정하는 바에 따른다. 다만, 주택 또는 도시와 관련된 특정한 사업을 원활히 추진하기 위하여 국토교통부장관이 따로 정하는 경우에는 그러하지 아니하다.',
 '제3조(기금의 운용·관리에 관한 사무) ① 「주택도시기금법(이하 "법"이라 한다)」 제10조제2항에 따라 주택도시기금(이하 "기금"이라 한다)의 운용 및 관리에 관한 사무는 주택도시보증공사(이하 "공사"라 한다)가 행한다.',
 '② 법 제10조제3항에 따라 공사는 위탁받은 사무의 일부를 국토교통부장관의 승인을 받아 금융기관 등에게 재위탁할 수 있으며, 국토교통부장관은 필요하다고 인정하는 경우 운용 및 관리에 관한 사

## Restruct

제\d편 > (제\d장) > (제\d절) > (제\d관) > 제\d조 

In [38]:
import re

### dictionary form

In [112]:
data_dic = {
    "title": data_list[4]
}
target_num = 0
temp_dic = {}
key_base = "title_level_0"
for d in data_list:
    target_text = "편장절관조"
    subtitle_match = re.match(f"제\d[{target_text}]", d)
    
    if subtitle_match:
        subtitle_name = subtitle_match.group()
        target_num = target_text.find(subtitle_name[-1]) 
        sub_title = d.split(subtitle_name)[1].strip()
        
        if target_num == 0:
            key_name = key_base 
            data_dic[before_subtitle_name] = temp_dic
            temp_dic = {}
            before_subtitle_name = subtitle_name
            
        else:
            key_name = key_name + "_" + str(target_num)

        # temp_dic[key_name] = subtitle_name
        # temp_dic[key_name+"_description"] = sub_title
        temp_dic[subtitle_name] = sub_title

data_dic[before_subtitle_name] = temp_dic       

In [113]:
pprint(data_dic)

{'title': '주택도시기금 운용 및 관리규정',
 '제1편': {'제1조': '(목적) 이 규정은 「주택도시기금법」 제3조에 따라 설치된 주택도시기금의 운용 및 관리에 관하여 필요한 사항을 '
                '규정함을 목적으로 한다.',
         '제1편': '총칙',
         '제2조': '(적용대상) 기금의 운용 및 관리업무는 관계법령에 규정한 것을 제외하고는 이 규정이 정하는 바에 따른다. '
                '다만, 주택 또는 도시와 관련된 특정한 사업을 원활히 추진하기 위하여 국토교통부장관이 따로 정하는 경우에는 '
                '그러하지 아니하다.',
         '제3조': '(기금의 운용·관리에 관한 사무) ① 「주택도시기금법(이하 "법"이라 한다)」 제10조제2항에 따라 '
                '주택도시기금(이하 "기금"이라 한다)의 운용 및 관리에 관한 사무는 주택도시보증공사(이하 "공사"라 한다)가 '
                '행한다.',
         '제4조': '(시행세칙) ① 이 규정의 시행에 필요한 구체적인 사항은 시행세칙으로 정하며, 시행세칙의 제정·변경 등 '
                '운용업무는 공사가 담당한다.'},
 '제2편': {'제1장': '기금운용계획 등',
         '제2장': '회계',
         '제2편': '기금의 관리',
         '제3장': '리스크 관리',
         '제4장': '성과 평가',
         '제5장': '자산유동화',
         '제5조': '(기금운용계획의 작성) ① 법 제10조제6항에 따른 기금운용계획은 다음 각 호로 구분하여 작성한다.',
         '제6조': '(기금운영비의 용도) 제5조제1항제6호의 기금운영비는 다음 각 호의 용도로 사용할 수 있다.',
         '제7조': '(기금운영비 등의 집행) ① 국토교통부장관은 기금운용계획이 정하는 금액의

In [114]:
data_list

['\x02捤獥\x00\x00\x00\x00\x02\x02汤捯\x00\x00\x00\x00\x02\x0b漠杳\x00\x00\x00\x00',
 '주택도시기금 운용 및 관리규정',
 '\x10慤桥\x00\x00\x00\x00\x10\x10潴景\x00\x00\x00\x00\x10[시행 2019. 12. 31.] [국토교통부훈령 제1271호, 2019. 12. 31, 일부개정.]',
 '氠瑢\x00\x00\x00\x00',
 '주택도시기금 운용 및 관리규정',
 '氠瑢\x00\x00\x00\x00',
 '漠杳\x00\x00\x00\x00\x0b 법제처',
 '- \x12湯慴\x00\x00\x00\x00\x12 / \x12湯慴\x00\x00\x00\x00\x12 -',
 '漠杳\x00\x00\x00\x00\x0b 국가법령정보센터',
 '국토교통부(총칙,성과평가), 044-201-3338, 3347',
 '국토교통부(기금운용계획,회계), 044-201-3339, 3346',
 '국토교통부(리스크,자산유동화), 044-201-3340, 3344',
 '제1편 총칙',
 '제1조(목적) 이 규정은 「주택도시기금법」 제3조에 따라 설치된 주택도시기금의 운용 및 관리에 관하여 필요한 사항을 규정함을 목적으로 한다.',
 '제2조(적용대상) 기금의 운용 및 관리업무는 관계법령에 규정한 것을 제외하고는 이 규정이 정하는 바에 따른다. 다만, 주택 또는 도시와 관련된 특정한 사업을 원활히 추진하기 위하여 국토교통부장관이 따로 정하는 경우에는 그러하지 아니하다.',
 '제3조(기금의 운용·관리에 관한 사무) ① 「주택도시기금법(이하 "법"이라 한다)」 제10조제2항에 따라 주택도시기금(이하 "기금"이라 한다)의 운용 및 관리에 관한 사무는 주택도시보증공사(이하 "공사"라 한다)가 행한다.',
 '② 법 제10조제3항에 따라 공사는 위탁받은 사무의 일부를 국토교통부장관의 승인을 받아 금융기관 등에게 재위탁할 수 있으며, 국토교통부장관은 필요하다고 인정하는 경우 운용 및 관리에 관한 사

### 편 단위 분리

In [116]:
index_list = []
for i, d in enumerate(data_list):
    target_text = "편"
    subtitle_match = re.match(f"제\d[{target_text}]", d)
    
    if subtitle_match:
        index_list.append(i)

In [124]:
pyun_list = [' '.join(data_list[index_list[x]:index_list[x+1]]) for x in range(len(index_list)-1)]

In [137]:
pyun_list[0]

'제1편 총칙 제1조(목적) 이 규정은 「주택도시기금법」 제3조에 따라 설치된 주택도시기금의 운용 및 관리에 관하여 필요한 사항을 규정함을 목적으로 한다. 제2조(적용대상) 기금의 운용 및 관리업무는 관계법령에 규정한 것을 제외하고는 이 규정이 정하는 바에 따른다. 다만, 주택 또는 도시와 관련된 특정한 사업을 원활히 추진하기 위하여 국토교통부장관이 따로 정하는 경우에는 그러하지 아니하다. 제3조(기금의 운용·관리에 관한 사무) ① 「주택도시기금법(이하 "법"이라 한다)」 제10조제2항에 따라 주택도시기금(이하 "기금"이라 한다)의 운용 및 관리에 관한 사무는 주택도시보증공사(이하 "공사"라 한다)가 행한다. ② 법 제10조제3항에 따라 공사는 위탁받은 사무의 일부를 국토교통부장관의 승인을 받아 금융기관 등에게 재위탁할 수 있으며, 국토교통부장관은 필요하다고 인정하는 경우 운용 및 관리에 관한 사무의 일부를 금융기관 등에게 직접 위탁할 수 있다. ③ 공사는 위탁사무의 수행을 위하여 제2항에 따라 기금 사무의 일부를 재위탁 또는 위탁받은 금융기관 등(이하 "기금재수탁자 등"이라 한다)에게 관련 자료의 제출 등 협조를 요청할 수 있으며, 특별한 사정이 없는 한 기금재수탁자 등은 이에 응하여야 한다. 제4조(시행세칙) ① 이 규정의 시행에 필요한 구체적인 사항은 시행세칙으로 정하며, 시행세칙의 제정·변경 등 운용업무는 공사가 담당한다. ② 공사는 시행세칙의 제정·변경이 기금재수탁자 등의 업무와 관련된 경우에는 공사 및 기금재수탁자 등의 임직원들로 구성된 주택도시기금 수탁기관협의체(이하 "협의체"라 한다)의 심의를 거쳐야 한다. ③ 공사가 시행세칙을 제정·변경한 때에는 지체없이 국토교통부장관의 승인을 받아야 한다. ④ 공사는 기금재수탁자 등과 협의하여 협의체의 운영에 관한 사항을 별도로 정할 수 있다.'

### title:description 단위

In [292]:
class titleDescriptionSeperator:
    def __init__(self, ):
        self.jang_name = None 
        self.jull_name = None
        self.gwan_name = None
        self.jo_name  = None

        self.jo_initiate = False
        self.jo_content = []

    def seperate(self, data_list):
        data_dic = {
            "doc_title": data_list[4]
        }
        
        for i, d in enumerate(data_list):
            target_text = "편장절관조"
            pyun_match = re.match(f"제\d+편", d)
            jang_match = re.match(f"제\d+장", d)
            jull_match = re.match(f"제\d+절", d)
            gwan_match = re.match(f"제\d+절", d)
            jo_match = re.match(f"제\d+조", d)
            
            if pyun_match:
                self.insert_jo(data_dic, data_list, i)
                self.pyun_name = pyun_match.group()
                data_dic[self.pyun_name] = {
                    "title" : self.seperate_by_space(d, self.pyun_name)
                }
                self.__init__()
        
            if jang_match:
                self.jang_name = jang_match.group()
                data_dic[self.get_name()] = {
                    "title" : self.seperate_by_space(d, self.jang_name)
                }
        
            if jull_match:
                self.jull_name = jull_match.group()
                data_dic[self.get_name()] = {
                    "title" : self.seperate_by_space(d, self.jull_name)
                }
        
            if gwan_match:
                self.gwan_name = gwan_match.group()
                data_dic[self.get_name()] = {
                    "title" : self.seperate_by_space(d, self.gwan_name)
                }
                
            if jo_match:
                self.insert_jo(data_dic, data_list, i)
                self.jo_name = jo_match.group()
                jo_search = re.search(r'\(([^)]+)', d)
                self.jo_content.append(d[jo_search.end()+1:].strip())
                data_dic[self.get_name()] = {
                    "title" :jo_search.group()[1:]
                }
        return data_dic

    def insert_jo(self, data_dic, data_list, i):
        if self.jo_initiate:
            self.jo_content.extend(data_list[self.jo_start+1:i])
            data_dic[self.get_name()]['content'] = ' '.join(self.jo_content)
        
        self.jo_start = i
        self.jo_content = []
        self.jo_initiate = True
        

    def seperate_by_space(self, d, name):
        return d.split(name)[-1].strip()
        
    def get_name(self,):
        default = f"{self.pyun_name}"
        if self.jang_name:
            default += f"_{self.jang_name}"
        if self.jull_name:
            default += f"_{self.jull_name}"
        if self.gwan_name:
            default += f"_{self.gwan_name}"
        if self.jo_name:
            default += f"_{self.jo_name}"
        return default

In [293]:
seperator = titleDescriptionSeperator()
data_dic = seperator.seperate(data_list)

In [294]:
pprint(data_dic)

{'doc_title': '주택도시기금 운용 및 관리규정',
 '제1편': {'title': '총칙'},
 '제1편_제1조': {'content': '이 규정은 「주택도시기금법」 제3조에 따라 설치된 주택도시기금의 운용 및 관리에 관하여 필요한 '
                        '사항을 규정함을 목적으로 한다.',
             'title': '목적'},
 '제1편_제2조': {'content': '기금의 운용 및 관리업무는 관계법령에 규정한 것을 제외하고는 이 규정이 정하는 바에 따른다. '
                        '다만, 주택 또는 도시와 관련된 특정한 사업을 원활히 추진하기 위하여 국토교통부장관이 따로 '
                        '정하는 경우에는 그러하지 아니하다.',
             'title': '적용대상'},
 '제1편_제3조': {'content': '① 「주택도시기금법(이하 "법"이라 한다)」 제10조제2항에 따라 주택도시기금(이하 "기금"이라 '
                        '한다)의 운용 및 관리에 관한 사무는 주택도시보증공사(이하 "공사"라 한다)가 행한다. ② 법 '
                        '제10조제3항에 따라 공사는 위탁받은 사무의 일부를 국토교통부장관의 승인을 받아 금융기관 등에게 '
                        '재위탁할 수 있으며, 국토교통부장관은 필요하다고 인정하는 경우 운용 및 관리에 관한 사무의 '
                        '일부를 금융기관 등에게 직접 위탁할 수 있다. ③ 공사는 위탁사무의 수행을 위하여 제2항에 따라 '
                        '기금 사무의 일부를 재위탁 또는 위탁받은 금융기관 등(이하 "기금재수탁자 등"이라 한다)에게 '
                        '관련 자료의 제출 등 협조를 요청할 수 있으며, 특별한 사정이 없는 한 기금재수탁자 

# Embedding Documents

## Create Collection

In [ ]:
import torch
import pandas as pd
import numpy as np

In [295]:
from sentence_transformers import SentenceTransformer
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
from pprint import pprint

In [6]:
MODEL = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'  # 내가 사용하고자 하는 임베딩 모델
COLLECTION_NAME = 'housingFundCorpus_db'  # 저장할 Vector Database의 Collection name
DIMENSION = 768  # 내가 사용하는 임베딩 모델의 dimension
LIMIT = 10  # topk와 같이 내가 검색했을 때 최대 몇개를 볼 것 인지
MILVUS_HOST = 'localhost' # 설치 Host
MILVUS_PORT = '19530' # 설치 Port 
INDEX_TYPE = "IVF_FLAT" # 다양한 Index 알고리즘들을 활용 가능 "HNSW" , "ANNOY" , "IVF_FLAT" , "IVF_SQ8" , "IVF_PQ"

In [549]:
# host와 port에 연결
connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)

# 내가 collection 이름이 겹치는 경우에는 삭제하겠다는 뜻
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

- varchar 최대 길이는 65,535

In [550]:
# 직접 밀어넣을 필드 & 스키마 정의하기
fields = [
    FieldSchema(name='doc_id', dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name='doc_title', dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name='date', dtype=DataType.VARCHAR, max_length=10),
    FieldSchema(name='content_num', dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name='content_title', dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name='content', dtype=DataType.VARCHAR, max_length=10000),
    FieldSchema(name='content_embeddings', dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]

schema = CollectionSchema(fields=fields, description='search text')
collection = Collection(name=COLLECTION_NAME, schema=schema)

# 매우 중요한 인덱스 부분
index_params = {
    'metric_type': 'L2',
    'index_type': INDEX_TYPE,
    'params':{"nlist":2048}
}

# colletion에서 색인 부분을 생성하는 부분을 명시하고 부르겠다는 뜻
collection.create_index(field_name="content_embeddings", index_params=index_params)
collection.load()

In [551]:
utility.list_collections()

['housingFundCorpus_db']

In [552]:
pprint(collection.describe())

{'aliases': [],
 'auto_id': False,
 'collection_id': 450067697748311687,
 'collection_name': 'housingFundCorpus_db',
 'consistency_level': 2,
 'description': 'search text',
 'enable_dynamic_field': False,
 'fields': [{'description': '',
             'field_id': 100,
             'is_primary': True,
             'name': 'doc_id',
             'params': {},
             'type': <DataType.INT64: 5>},
            {'description': '',
             'field_id': 101,
             'name': 'doc_title',
             'params': {'max_length': 100},
             'type': <DataType.VARCHAR: 21>},
            {'description': '',
             'field_id': 102,
             'name': 'date',
             'params': {'max_length': 10},
             'type': <DataType.VARCHAR: 21>},
            {'description': '',
             'field_id': 103,
             'name': 'content_num',
             'params': {'max_length': 50},
             'type': <DataType.VARCHAR: 21>},
            {'description': '',
             '

## Convert to Embeddings

In [313]:
from towhee import pipe, ops, DataCollection

In [514]:
EMBEDDING_MODEL = 'facebook/dpr-ctx_encoder-single-nq-base'

In [511]:
p = (
    pipe.input('text')
        .map('text', 'vec', ops.text_embedding.dpr(model_name=EMBEDDING_MODEL))
        .output('text', 'vec')
)

DataCollection(p('Hello, world.')).show()

text,vec
"Hello, world.","[-0.41551235, 0.32320973, 0.198512, ...] shape=(768,)"


Towhee supports reading df data through the from_df interface, and then we need to convert the title_vector column in the data to a two-dimensional list in float format, and then insert all the fields into Milvus, each field inserted into Milvus corresponds to one Collection fields created earlier.

In [310]:
df = pd.DataFrame(data_dic).T.dropna()

In [311]:
df

,title,content
doc_title,주택도시기금 운용 및 관리규정,주택도시기금 운용 및 관리규정
제1편_제1조,목적,이 규정은 「주택도시기금법」 제3조에 따라 설치된 주택도시기금의 운용 및 관리에 관...
제1편_제2조,적용대상,기금의 운용 및 관리업무는 관계법령에 규정한 것을 제외하고는 이 규정이 정하는 바에...
제1편_제3조,기금의 운용·관리에 관한 사무,"① 「주택도시기금법(이하 ""법""이라 한다)」 제10조제2항에 따라 주택도시기금(이하..."
제1편_제4조,시행세칙,"① 이 규정의 시행에 필요한 구체적인 사항은 시행세칙으로 정하며, 시행세칙의 제정·..."
...,...,...
제3편_제3장_제2절_제2절_제57조,지원대상의 선정,"① 공사는 사업의 수익성, 안정성 및 사업조건의 적정성 등을 종합적으로 심사하여 신..."
제3편_제3장_제2절_제2절_제58조,약정,① 공사는 제57조제1항의 심사를 거쳐 기금 지원이 결정된 경우에는 그 지원방식에 ...
제3편_제3장_제2절_제2절_제59조,사후관리,① 공사는 도시재생사업에 대한 기금의 지원 시부터 반환 시까지 각각의 지원방식에 따...
제4편_제60조,업무처리기준 등,공사 및 기금재수탁자 등은 이 규정 및 제4조의 시행세칙이 정하는 범위 내에서 기금...


In [473]:
# from towhee import AutoPipes, AutoConfig

# # get the built-in sentence_similarity pipeline
# config = AutoConfig.load_config('sentence_embedding')
# config.model = 'paraphrase-albert-small-v2'
# config.device = 0
# sentence_embedding = AutoPipes.pipeline('sentence_embedding', config=config)

# # # generate embedding for one sentence
# # embedding = sentence_embedding('how are you?').get()

# # batch generate embeddings for multi-sentences
# embeddings = sentence_embedding.batch(df.content.tolist())
# embeddings = [e.get() for e in embeddings]

In [528]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer

tokenizer = DPRContextEncoderTokenizer.from_pretrained(EMBEDDING_MODEL)
model = DPRContextEncoder.from_pretrained(EMBEDDING_MODEL)

input_ids = [tokenizer.encode(x, padding=True, truncation=True, return_tensors="pt") for x in df.content.tolist()]
embeddings = [model(x).pooler_output for x in input_ids]
embeddings = [x[0] for x in embeddings]
embeddings = [x.detach().numpy() for x in embeddings]

In [545]:
df['embeddings'] = embeddings

In [555]:
df

,title,content,embeddings
doc_title,주택도시기금 운용 및 관리규정,주택도시기금 운용 및 관리규정,"[-0.33567426, 0.33105344, 0.0750302, -0.764138..."
제1편_제1조,목적,이 규정은 「주택도시기금법」 제3조에 따라 설치된 주택도시기금의 운용 및 관리에 관...,"[-0.41396818, 0.09340677, 0.43317145, -0.66389..."
제1편_제2조,적용대상,기금의 운용 및 관리업무는 관계법령에 규정한 것을 제외하고는 이 규정이 정하는 바에...,"[-0.333913, 0.14708404, 0.077873394, -0.404807..."
제1편_제3조,기금의 운용·관리에 관한 사무,"① 「주택도시기금법(이하 ""법""이라 한다)」 제10조제2항에 따라 주택도시기금(이하...","[-0.34027332, 0.16220723, 0.2442497, -0.293144..."
제1편_제4조,시행세칙,"① 이 규정의 시행에 필요한 구체적인 사항은 시행세칙으로 정하며, 시행세칙의 제정·...","[-0.3580566, 0.2050047, 0.22397837, -0.4959316..."
...,...,...,...
제3편_제3장_제2절_제2절_제57조,지원대상의 선정,"① 공사는 사업의 수익성, 안정성 및 사업조건의 적정성 등을 종합적으로 심사하여 신...","[-0.47826055, 0.15056068, 0.17028502, -0.28672..."
제3편_제3장_제2절_제2절_제58조,약정,① 공사는 제57조제1항의 심사를 거쳐 기금 지원이 결정된 경우에는 그 지원방식에 ...,"[-0.35148728, 0.19578853, 0.12163217, -0.52609..."
제3편_제3장_제2절_제2절_제59조,사후관리,① 공사는 도시재생사업에 대한 기금의 지원 시부터 반환 시까지 각각의 지원방식에 따...,"[-0.35049823, 0.21723364, 0.10572036, -0.52162..."
제4편_제60조,업무처리기준 등,공사 및 기금재수탁자 등은 이 규정 및 제4조의 시행세칙이 정하는 범위 내에서 기금...,"[-0.65172356, 0.16517003, 0.24464312, -0.57346..."


## Data to Milvus

In [556]:
rdf = df.iloc[1:].reset_index().reset_index()

columns = ["doc_id", "doc_title", "date", "content_num", "content_title", "content", "content_embeddings"]

rdf = rdf.rename(columns={"level_0":"doc_id", "title":"content_title", "index": "content_num", "embeddings": "content_embeddings", })

rdf['doc_title'] = data_dic['doc_title']
rdf['date'] = open_date
rdf = rdf[columns]

In [557]:
rdf.head()

,doc_id,doc_title,date,content_num,content_title,content,content_embeddings
0,0,주택도시기금 운용 및 관리규정,2019.12.31,제1편_제1조,목적,이 규정은 「주택도시기금법」 제3조에 따라 설치된 주택도시기금의 운용 및 관리에 관...,"[-0.41396818, 0.09340677, 0.43317145, -0.66389..."
1,1,주택도시기금 운용 및 관리규정,2019.12.31,제1편_제2조,적용대상,기금의 운용 및 관리업무는 관계법령에 규정한 것을 제외하고는 이 규정이 정하는 바에...,"[-0.333913, 0.14708404, 0.077873394, -0.404807..."
2,2,주택도시기금 운용 및 관리규정,2019.12.31,제1편_제3조,기금의 운용·관리에 관한 사무,"① 「주택도시기금법(이하 ""법""이라 한다)」 제10조제2항에 따라 주택도시기금(이하...","[-0.34027332, 0.16220723, 0.2442497, -0.293144..."
3,3,주택도시기금 운용 및 관리규정,2019.12.31,제1편_제4조,시행세칙,"① 이 규정의 시행에 필요한 구체적인 사항은 시행세칙으로 정하며, 시행세칙의 제정·...","[-0.3580566, 0.2050047, 0.22397837, -0.4959316..."
4,4,주택도시기금 운용 및 관리규정,2019.12.31,제2편_제1장_제5조,기금운용계획의 작성,① 법 제10조제6항에 따른 기금운용계획은 다음 각 호로 구분하여 작성한다. 1. ...,"[-0.15092611, 0.12241694, 0.27259725, -0.44828..."


In [558]:
from towhee import ops, pipe, DataCollection

In [559]:
insert_pipe = (pipe.input('df')
                   .flat_map('df', 'data', lambda df: df.values.tolist())
                   .map('data', 'res', ops.ann_insert.milvus_client(host=MILVUS_HOST, 
                                                                    port=MILVUS_PORT,
                                                                    collection_name=COLLECTION_NAME))
                   .output('res')
)

%time _ = insert_pipe(rdf)

CPU times: user 51.9 ms, sys: 12.9 ms, total: 64.9 ms
Wall time: 208 ms


In [561]:
collection.load()
collection.num_entities

0

# Search embedding title

In [562]:
import numpy as np

search_pipe = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('doc_id', 'score'), ops.ann_search.milvus_client(host=MILVUS_HOST, 
                                                                                   port=MILVUS_PORT,
                                                                                   collection_name=COLLECTION_NAME))  
                    .output('query', 'doc_id', 'score')
               )

In [563]:
res = search_pipe('funny python demo')
DataCollection(res).show()

query,doc_id,score
funny python demo,24,138.8067626953125
funny python demo,5,144.09185791015625
funny python demo,47,144.50384521484375
funny python demo,18,145.25360107421875
funny python demo,50,145.6643524169922


## Search multi text in Milvus

In [564]:
res = search_pipe.batch(['funny python demo', 'AI in data analysis'])
for re in res:
    DataCollection(re).show()

query,doc_id,score
funny python demo,24,138.8067626953125
funny python demo,5,144.09185791015625
funny python demo,47,144.50384521484375
funny python demo,18,145.25360107421875
funny python demo,50,145.6643524169922


query,doc_id,score
AI in data analysis,24,138.35169982910156
AI in data analysis,47,144.11634826660156
AI in data analysis,18,145.61318969726562
AI in data analysis,50,145.94979858398438
AI in data analysis,5,146.0421142578125


## Search text and return multi fields

In [565]:
search_pipe1 = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('doc_id', 'score', 'doc_title', 'content_num', 'content_title', 'content'), ops.ann_search.milvus_client(host=MILVUS_HOST, 
                                                                                   port=MILVUS_PORT,
                                                                                   collection_name=COLLECTION_NAME,
                                                                                   output_fields=['doc_title', 'content_num', 'content_title', 'content']))  
                    .output('query', 'doc_id', 'score', 'doc_title', 'content_num', 'content_title', 'content')
               )

In [568]:
res = search_pipe1('기금운영비 용도가 뭐야?')
DataCollection(res).show()

query,doc_id,score,doc_title,content_num,content_title,content
기금운영비 용도가 뭐야?,24,129.412353515625,주택도시기금 운용 및 관리규정,제3편_제2장_제1절_제1절_제25조,융자형식,자금의 융자는 증서대출을 원칙으로 한다.
기금운영비 용도가 뭐야?,47,134.6744842529297,주택도시기금 운용 및 관리규정,제3편_제2장_제2절_제2절_제48조,출자대금 실행,공사는 출자승인 후 사업약정에 따라 출자대금을 실행하여야 한다.
기금운영비 용도가 뭐야?,50,136.7135009765625,주택도시기금 운용 및 관리규정,제3편_제2장_제2절_제2절_제51조,출자대금 수납,"공사는 청산 등 임대리츠 등에 대한 정산이 완료된 때 반환금액, 일시 등에 관한 사항을 국토교통부장관에게 보고하고, 반환금액이 국토교통부장관이 지정한 계좌에 수납되도록 조치하여야 한다."
기금운영비 용도가 뭐야?,18,137.0990447998047,주택도시기금 운용 및 관리규정,제3편_제1장_제19조,기금의 용도,"① 기금은 법 제9조, 「주택도시기금법 시행령」 제13조 및 제15조에 규정한 용도에 따라 자금을 구분하여 운용한다. ② 자금의 종류, 각 자금별 지원대상 및 지원한도, 지원기간 등과 관련하여 이 규정에서 정하지 않은 사항은 기금..."
기금운영비 용도가 뭐야?,45,137.26025390625,주택도시기금 운용 및 관리규정,제3편_제2장_제2절_제2절_제46조,출자승인,"① 공사는 출자의사를 결정하기 위하여 관계 전문가로 구성된 투자심사위원회를 구성하여야 한다. ② 제1항의 위원회에서 출자여부가 최종 결정되면, 그 결과를 국토교통부장관에게 보고하고, 출자신청인에게 통지하여야 한다."
